# OSM Geohash File Combination

In [1]:
## Basic stuff
#%load_ext autoreload
#%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


## Python Version
import sys
print("Python: {0}".format(sys.version))


## Install
import shapefile
import geohash
from timeUtils import clock, elapsed
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from random import uniform
from fsUtils import isFile
from ioUtils import showSize, saveJoblib, saveFile
from geoUtils import *
from geospatialUtils import saveGeoData, getBB, rmZipDir, getGeos
import pickle
from glob import glob
from os.path import basename,splitext,join
from collections import Counter
from string import ascii_uppercase

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.6.5 |Anaconda custom (x86_64)| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2018-11-28 09:33:09.880511


In [2]:
# Global Params
basedir="/Users/tgadf/Downloads/osm"
prec=7
mainkey="OSM"
letters=ascii_uppercase
_, _ = clock("Last Run")

Current Time is Wed Nov 28, 2018 09:33:10 for Last Run


# Combine Geohash Data

In [6]:
import pickle
prec=7
from glob import glob
from os.path import basename,splitext, join
from collections import Counter

#mtypes   = set([splitext(basename(x))[0].split('-')[1] for x in vals])
mtypes  = ["traffic", "pofw", "pois", "transport", "buildings"]
mtypes  = ["buildings"]
mtypes  = ["pofw"]
mtypes  = ["traffic"]
mtypes  = ["transport"]
mtypes  = ["pois"]
mtypes  = ["traffic", "pofw", "pois", "transport"]

osmdata = {}
#osmdata["traffic"] = ['parking', 'fuel']
#osmdata["transport"] = ['bus', 'tram', 'rail', 'ferry', 'taxi']
#osmdata["pofw"]      = ['christian', 'buddhist', 'jewish', 'muslim', 'other']
#osmdata["poi"]       = ['recreation', 'sport', 'public', 'business', 'fastfood', 'restaurant', 'school', 'attraction', 'municipal', 'lodging', 'manufacturing', 'grocery', 'entertainment', 'auto', 'medical', 'religious', 'college']
#osmdata["water"]     = ['stream', 'water', 'river', 'wetland', 'drain', 'canal', 'reservoir', 'glacier', 'dock']
#osmdata["landuse"]   = ['residential', 'farm', 'grass', 'commercial', 'meadow', 'park', 'forest', 'vineyard', 'orchard', 'industrial', 'scrub', 'retail', 'heath', 'cemetery', 'nature_reserve', 'recreation_ground', 'quarry', 'military', 'allotments']
#osmdata["place"]     = ['hamlet', 'locality', 'city', 'island', 'village', 'town', 'suburb', 'county', 'farm', 'region']
osmdata["natural"]   = ['tree', 'peak', 'spring', 'beach', 'cliff', 'cave_entrance', 'glacier', 'volcano']

#mtypes  = ["water"]
#mtypes  = ["landuse"]
#mtypes  = ["place"]
#mtypes  = ["natural"]

print("Mtypes --> {0}".format(osmdata.keys()))
for mtype,categories in osmdata.items():
    print("  Categories --> {0}".format(categories))
    for category in categories:
        vals    = glob(join(basedir, "*-{0}-{1}-{2}-geos.p".format(mtype, category, prec)))        
        states  = set([splitext(basename(x))[0].split('-{0}'.format(mtype))[0] for x in vals])
        print("    States --> {0}".format(states))
        print("    Found  --> {0} files.\n".format(len(states)))

        print("============== {0} ==============".format(mtype))
        print("  ============ {0} ============".format(category))
        if mtype in ["water", "landuse"]:
            fnames = letters[:5]
        elif mtype == "place":
            fnames = letters[:3]
        else:
            fnames = letters[:1]
            
        fnames = letters[:1]
        for let in range(len(fnames)):
            ngeosMtypes = 0
            ngeos = 0
            geomap  = {}
            records = {}
            cntr    = Counter()
            for istate,state in enumerate(states):
                if istate % len(fnames) != let:
                    continue
                    
                base = "{0}-{1}-{2}-{3}".format(state, mtype, category, prec)
                recfiles = glob(join(basedir, "{0}-data.p".format(base)))
                geofiles = glob(join(basedir, "{0}-geos.p".format(base)))

                #print("  Found {0} and {1} rec/geo files for {2}".format(len(recfiles), len(geofiles), mtype))
                for i in range(len(geofiles)):
                    recfile = recfiles[i]
                    geofile = geofiles[i]

                    try:
                        recdata = pickle.load(open(recfile, "rb"))
                        geodata = pickle.load(open(geofile, "rb"))
                        #print("Opened {0}".format(recfile))
                        #print("Opened {0}".format(geofile))
                    except:
                        print("Could not open {0}".format(geofile))
                        1/0
                        continue

                    #print(geodata.keys())
                    #continue
                    ngeos = 0
                    for geoid,geos in geodata.items():
                        ngeos += len(geos)
                        ngeosMtypes += len(geos)
                        for geo in geos:
                            if geomap.get(geo) is None:
                                geomap[geo] = Counter()
                            #print(geo,mtype,geoid)
                            #print(geoid)
                            #print(recdata.keys())
                            if recdata.get(geoid) is not None:
                                try:
                                    #print(geoid)
                                    #print(recdata[geoid])
                                    #name = recdata[geoid]['Name']
                                    #print(name)
                                    name = geoid
                                except:
                                    name = geoid
                                    #print(name)
                            else:
                                name = geoid
                            cntr[name] += 1
                            geomap[geo][name] += 1

                    for geoid,rec in recdata.items():
                        records[geoid] = rec['Name']
                        break

                print("{0: <20}\t{1: <20}\t{2: <12}\t{3: <12}\t---> {4} <---".format(mtype,state,ngeos,ngeosMtypes,len(geomap)))            
            print("{0: <20}\t{1: <20}\t{2: <12}\t{3: <12}\t---> {4} <---".format(mtype,"","",ngeosMtypes,len(geomap)))

            for k,v in cntr.most_common():
                k = str(k)
                if k.find(" ") != -1 or k.find(";") != -1 or k.find("_") != -1 or k.find(",") != -1 or k[0].isupper():
                    print("buildingConv[\"{0}\"] = \"{1}\"".format(k, k))
                    found = True

            for geo in geomap.keys():
                geomap[geo] = dict(geomap[geo])


            if len(fnames) == 1:
                fname="geomap-{0}-{1}-{2}-OSM.p".format(prec, mtype.title(), category.title())
            else:
                fname="geomap-{0}-{1}-{2}-{3}-OSM.p".format(prec, mtype.title(), category.title(), fnames[let])
            print("Writing {0}".format(fname))
            saveFile(idata=geomap, ifile=fname)

            if len(fnames) == 1:
                fname="georec-{0}-{1}-{2}-OSM.p".format(prec, mtype.title(), category.title())
            else:
                fname="georec-{0}-{1}-{2}-{3}-OSM.p".format(prec, mtype.title(), category.title(), fnames[let])
            print("Writing {0}".format(fname))
            saveFile(idata=records, ifile=fname)

Mtypes --> dict_keys(['natural'])
  Categories --> ['tree', 'peak', 'spring', 'beach', 'cliff', 'cave_entrance', 'glacier', 'volcano']
    States --> {'iowa', 'missouri', 'wyoming', 'new-jersey', 'new-york', 'pennsylvania', 'colorado', 'alaska', 'alabama', 'minnesota', 'florida', 'mississippi', 'tennessee', 'arizona', 'kansas', 'wisconsin', 'maine', 'georgia', 'california', 'idaho', 'louisiana', 'new-mexico', 'north-carolina', 'massachusetts', 'texas', 'indiana', 'nevada', 'vermont', 'michigan', 'montana', 'puerto-rico', 'north-dakota', 'south-carolina', 'kentucky', 'rhode-island', 'connecticut', 'maryland', 'washington', 'arkansas', 'hawaii', 'delaware', 'nebraska', 'illinois', 'new-hampshire', 'oregon', 'south-dakota', 'utah', 'oklahoma', 'ohio', 'west-virginia', 'virginia', 'district-of-columbia'}
    Found  --> 52 files.

============== natural ==============
  ============ tree ============
natural             	iowa                	887         	887         	---> 887 <---
natural  

natural             	indiana             	354         	43600       	---> 43442 <---
natural             	nevada              	1819        	45419       	---> 45237 <---
natural             	vermont             	1267        	46686       	---> 46499 <---
natural             	michigan            	287         	46973       	---> 46782 <---
natural             	montana             	3390        	50363       	---> 50078 <---
natural             	puerto-rico         	301         	50664       	---> 50379 <---
natural             	north-dakota        	306         	50970       	---> 50685 <---
natural             	south-carolina      	307         	51277       	---> 50967 <---
natural             	kentucky            	1114        	52391       	---> 52079 <---
natural             	rhode-island        	235         	52626       	---> 52311 <---
natural             	connecticut         	1185        	53811       	---> 53476 <---
natural             	maryland            	260         	54071       	---> 537

natural             	texas               	2403        	50589       	---> 50366 <---
natural             	indiana             	547         	51136       	---> 50913 <---
natural             	nevada              	123         	51259       	---> 51034 <---
natural             	vermont             	120         	51379       	---> 51154 <---
natural             	michigan            	5520        	56899       	---> 56674 <---
natural             	montana             	18          	56917       	---> 56692 <---
natural             	puerto-rico         	1005        	57922       	---> 57697 <---
natural             	north-dakota        	39          	57961       	---> 57735 <---
natural             	south-carolina      	2241        	60202       	---> 59896 <---
natural             	kentucky            	19          	60221       	---> 59915 <---
natural             	rhode-island        	932         	61153       	---> 60829 <---
natural             	connecticut         	664         	61817       	---> 614

natural             	alaska              	570745      	596346      	---> 596312 <---
natural             	                    	            	596346      	---> 596312 <---
Writing geomap-7-Natural-Glacier-OSM.p
  --> This file is 5.7MB.
Writing georec-7-Natural-Glacier-OSM.p
  --> This file is 32B.
    States --> {'washington', 'arizona', 'nevada', 'wyoming', 'oregon', 'hawaii', 'california', 'utah', 'alaska', 'idaho', 'new-mexico', 'florida'}
    Found  --> 12 files.

============== natural ==============
  ============ volcano ============
natural             	washington          	6           	6           	---> 6 <---
natural             	arizona             	83          	89          	---> 89 <---
natural             	nevada              	1           	90          	---> 90 <---
natural             	wyoming             	1           	91          	---> 91 <---
natural             	oregon              	64          	155         	---> 155 <---
natural             	hawaii              	146    

In [3]:
x = [('residential', 199889), ('farm', 26847), ('grass', 18283), ('commercial', 17961), ('meadow', 16291), ('park', 13203), ('forest', 12230), ('vineyard', 11816), ('orchard', 10650), ('industrial', 10119), ('scrub', 8833), ('retail', 5567), ('heath', 1927), ('cemetery', 982), ('nature_reserve', 961), ('recreation_ground', 742), ('quarry', 400), ('military', 251), ('allotments', 102)]

In [4]:
print(", ".join([t[0] for t in x]))

residential, farm, grass, commercial, meadow, park, forest, vineyard, orchard, industrial, scrub, retail, heath, cemetery, nature_reserve, recreation_ground, quarry, military, allotments


In [ ]:
x